In [1]:
import sys
import os
import nest_asyncio
nest_asyncio.apply()

# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from jarjarquant import Jarjarquant
jjq = Jarjarquant()

Error 10275, reqId -1: Positions info is not available for account(s): U12104351 until the application is finished and approved.


In [5]:
from ib_async import IB, util, Index
ib = IB()

# Connect to the IB Gateway or TWS
await ib.connectAsync('127.0.0.1', 7496, clientId=1)

contract = Index('SPX', 'CBOE')

# Request historical implied volatility data
bars = await ib.reqHistoricalDataAsync(
    contract,
    endDateTime='',  # current time
    durationStr='68 Y',  # Duration of data, e.g., '1 M' for 1 month
    barSizeSetting='1 day',  # Bar size, e.g., '1 day'
    whatToShow='TRADES',
    useRTH=True,
    formatDate=1
)

# Convert bars to a DataFrame and display
df = util.df(bars)
# Rename 'open', 'high', 'low', 'close' columns to 'Open', 'High', 'Low', 'Close'
# df.rename(columns={'open': 'Open', 'high': 'High',
#             'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

# Disconnect from IB
ib.disconnect()

Error 10275, reqId -1: Positions info is not available for account(s): U12104351 until the application is finished and approved.


In [6]:
df

,date,open,high,low,close,volume,average,barCount
0,2004-03-04,1150.61,1154.98,1149.80,1154.57,0.0,0.0,692
1,2004-03-05,1151.00,1163.23,1148.75,1157.34,0.0,0.0,1806
2,2004-03-08,1157.75,1160.12,1147.02,1147.02,0.0,0.0,1133
3,2004-03-09,1146.59,1146.81,1136.86,1140.67,0.0,0.0,1439
4,2004-03-10,1141.61,1142.05,1122.50,1124.10,0.0,0.0,1413
...,...,...,...,...,...,...,...,...
5284,2025-03-04,5811.98,5865.08,5732.59,5778.15,0.0,0.0,22936
5285,2025-03-05,5781.36,5860.59,5742.35,5842.63,0.0,0.0,22876
5286,2025-03-06,5785.87,5812.08,5711.64,5738.52,0.0,0.0,22874
5287,2025-03-07,5726.01,5783.01,5666.29,5770.20,0.0,0.0,22900


In [3]:
# Use the get_tws_ticker method from data gatherer
# Empty end date implies fetching data upto the last available date
jjq.data_gatherer.get_tws_ticker(ticker='BAC', exchange='SMART', currency='USD', end_date='', duration='1 D', bar_size='1 min', what_to_show='TRADES')

Error 10275, reqId -1: Positions info is not available for account(s): U12104351 until the application is finished and approved.


,date,Open,High,Low,Close,Volume,average,barCount
0,2025-03-06 09:30:00-05:00,41.59,41.79,41.49,41.68,916490.0,41.735,1777
1,2025-03-06 09:31:00-05:00,41.68,41.72,41.53,41.56,187536.0,41.627,626
2,2025-03-06 09:32:00-05:00,41.55,41.59,41.45,41.47,155585.0,41.521,631
3,2025-03-06 09:33:00-05:00,41.47,41.48,41.18,41.21,298652.0,41.272,1208
4,2025-03-06 09:34:00-05:00,41.22,41.35,41.17,41.32,183189.0,41.271,841
...,...,...,...,...,...,...,...,...
385,2025-03-06 15:55:00-05:00,41.47,41.50,41.44,41.48,194793.0,41.469,661
386,2025-03-06 15:56:00-05:00,41.48,41.49,41.40,41.41,207376.0,41.443,504
387,2025-03-06 15:57:00-05:00,41.41,41.45,41.37,41.39,433668.0,41.394,808
388,2025-03-06 15:58:00-05:00,41.40,41.40,41.34,41.36,411203.0,41.371,1076


In [5]:
# Get random price samples
# Valid Bar Sizes:
# 1 secs 5 secs	10 secs	15 secs	30 secs
# 1 min	2 mins	3 mins	5 mins	10 mins	15 mins	20 mins	30 mins
# 1 hour 2 hours 3 hours 4 hours 8 hours
# 1 day
# 1 week
# 1 month
data = jjq.data_gatherer.get_random_price_samples_tws(num_tickers_to_sample=1, years_in_sample=1, bar_size='1 day')
data[0].head()  # Display the first few rows of the sampled data

,date,Open,High,Low,Close,Volume,average,barCount
0,2018-01-17,331.46,335.74,330.32,334.86,620155.0,333.953,4069
1,2018-01-18,334.11,335.92,332.99,333.00,539609.0,334.232,3333
2,2018-01-19,334.24,334.89,331.78,332.00,937726.0,332.837,5496
3,2018-01-22,331.77,332.70,328.97,329.00,1043736.0,330.603,7755
4,2018-01-23,329.99,331.15,328.51,328.70,687909.0,329.634,4754


In [ ]:
# Override years_in_sample to get samples less than 1 year by specifying a custom duration
# Duration formnat:
# "<integer> <duration_unit>"
# Valid durations:
# S	Seconds
# D	Day
# W	Week
# M	Month
# Y	Year
# Ensure years_in_sample is an integer and greater than the specified duration - not specifying years_in_sample will handle both these risks
data = jjq.data_gatherer.get_random_price_samples_tws(num_tickers_to_sample=1, bar_size='5 mins', duration='1 D')
data[0].head()  # Display the first few rows of the sampled data

,date,Open,High,Low,Close,Volume,average,barCount
0,2024-04-04 09:30:00-04:00,40.59,40.70,40.35,40.45,1548201.0,40.504,4345
1,2024-04-04 09:35:00-04:00,40.45,40.71,40.43,40.52,875790.0,40.552,3551
2,2024-04-04 09:40:00-04:00,40.52,40.76,40.50,40.71,737174.0,40.655,3094
3,2024-04-04 09:45:00-04:00,40.71,40.75,40.53,40.65,647403.0,40.661,3570
4,2024-04-04 09:50:00-04:00,40.64,40.66,40.41,40.44,615366.0,40.515,2954


#### Using the EODHD API to fetch data

In [1]:
import polars as pl

# Empty DataFrame
empty_df = pl.DataFrame()
print(empty_df.is_empty())  # True
print(empty_df.height == 0)  # True

# DataFrame with columns but no rows
empty_with_schema = pl.DataFrame({"a": [], "b": []})
print(empty_with_schema.is_empty())  # True

# Non-empty DataFrame
df = pl.DataFrame({"a": [1, 2], "b": [3, 4]})
print(df.is_empty())  # False

True
True
True
False
